<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.1</h1>

Perform a depth-first search on the following graph; whenever there’s a choice of  vertices, pick the one that is alphabetically first. Classify each edge as a tree edge  or back edge, and give the pre and post number of each vertex.

-----

Skip

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.2</h1>

Perform depth-first search on each of the following graphs; whenever there’s a  choice of vertices, pick the one that is alphabetically first. Classify each edge as a  tree edge, forward edge, back edge, or cross edge, and give the pre and post  number of each vertex. 

-----

Skip

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.7</h1>


A bipartite graph is a graph G = (V, E) whose vertices can be partitioned into  two sets (V = V1 ∪ V2 and V1 ∩ V2 = ∅) such that there are no edges between  vertices in the same set (for instance, if u, v ∈ V1, then there is no edge between  u and v). 

(a) Give a linear-time algorithm to determine whether an undirected graph is  bipartite.  

(b) There are many other ways to formulate this property. For instance, an  undirected graph is bipartite if and only if it can be colored with just two  colors.  Prove the following formulation: an undirected graph is bipartite if and  only if it contains no cycles of odd length.  

(c) At most how many colors are needed to color in an undirected graph  with exactly one odd-length cycle? 

-----

a) 在图上进行 DFS，并将在DFS 树上位于不同层的顶点交替地着上红黑二色，使
得每条tree edge的两个顶点颜色都不相同。如果遇到一条non-tree edge (u,v)，
且有顶点u,v的颜色相同，则返回假，否则返回真。时间复杂度O(V + E)。

b) 根据上面的算法可以知道，一个图为二分的一个充要条件是对于每条non-tree
edge (u,v)，有顶点u,v的颜色不相同，也即是在 DFS 树上顶点u,v要间隔奇
数层。而环的长度为间隔层数加1，为偶数。于是得证。

c) 三种颜色。

-----
<font color='red'><b>OA solution</b></font>

## (a) Algorithm

Represent the undirected input graph $G=(V,E)$ in adjacency-list form.  
Introduce a Boolean variable $x_v$ for each vertex $v \in V$.  
Interpret $x_v = \text{true}$ as “$v$ is colored with color 1 (in $V_1$)” and $x_v = \text{false}$ as “$v$ is colored with color 2 (in $V_2$).$”  

For every edge $\{u, v\} \in E$, we must enforce that its endpoints receive **different** colors, i.e. $x_u \ne x_v$.  
The XOR constraint $x_u \ne x_v$ can be expressed as the 2-CNF:

$$
(x_u \lor x_v) \land (\lnot x_u \lor \lnot x_v)
$$

Build the conjunction of these two clauses for every edge in $E$.

Now construct the **implication graph** $G_{\text{imp}}$ with literals $\{x_v, \lnot x_v : v \in V\}$, where each clause $(a \lor b)$ contributes the implications $(\lnot a \rightarrow b)$ and $(\lnot b \rightarrow a)$.  
Concretely, for each edge $\{u, v\}$, add the implications:

$$
\lnot x_u \rightarrow x_v, \quad \lnot x_v \rightarrow x_u, \quad
x_u \rightarrow \lnot x_v, \quad x_v \rightarrow \lnot x_u
$$

Use the **2-SAT algorithm** (black box) to check satisfiability:
- Compute strongly connected components (SCCs) of $G_{\text{imp}}$.
- If any variable $x_v$ and its negation $\lnot x_v$ are in the **same SCC**, the formula is unsatisfiable.
- Otherwise, it is satisfiable. Assign truth values by processing SCCs in reverse topological order.

If satisfiable, the graph is **bipartite**; assign vertices to sets:
- $v \in V_1$ if $x_v = \text{true}$
- $v \in V_2$ if $x_v = \text{false}$

If unsatisfiable, the graph is **not bipartite**.

---

## (b) Justification of Correctness

### CNF encodes bipartiteness
For each edge $\{u,v\}$, bipartiteness requires $u$ and $v$ to be in opposite sets.  
The clause pair $(x_u \lor x_v) \land (\lnot x_u \lor \lnot x_v)$ enforces $x_u \oplus x_v = \text{true}$, so any satisfying assignment corresponds to a valid 2-coloring.  
Conversely, any valid bipartition satisfies all these constraints.  
Hence, the 2-CNF is satisfiable **iff** $G$ is bipartite.

### Odd cycles as obstructions
- **If $G$ is bipartite**: all cycles alternate between $V_1$ and $V_2$, so every cycle has **even** length — no odd cycles exist.
- **If $G$ contains an odd cycle** $C = (v_0, v_1, \ldots, v_{2k}, v_{2k+1}=v_0)$:  
  Each edge $\{v_i, v_{i+1}\}$ enforces $x_{v_{i+1}} = \lnot x_{v_i}$.  
  Chaining around the odd cycle gives $x_{v_0} = \lnot x_{v_0}$, meaning $x_{v_0}$ and $\lnot x_{v_0}$ belong to the same SCC — making the 2-CNF unsatisfiable.

If the graph has **no odd cycle**, BFS parity coloring gives a satisfying assignment to all $x_v$.  
Thus, the formula is satisfiable **iff** $G$ is bipartite.

---

## (c) Colors Needed with Exactly One Odd Cycle

At most **three** colors are needed.

An odd cycle requires at least 3 colors to color its vertices.  
If the graph has exactly one odd cycle:
1. Remove any vertex $x$ on that odd cycle — the remaining graph has no odd cycles, so it is 2-colorable.
2. Color the remaining vertices with two colors.
3. Assign $x$ a **third** color.

Thus, the graph is **3-colorable** but not 2-colorable.  
The answer is **3 colors**.

---

## (d) Runtime Analysis

- **CNF / Implication Graph Construction:** For each edge, a constant number of clauses and implications are added → $O(n + m)$.
- **2-SAT (SCC Computation):** Running Tarjan’s or Kosaraju’s algorithm on $G_{\text{imp}}$ with $O(n)$ literals and $O(m)$ edges → $O(n + m)$.
- **Assignment Extraction / Coloring:** $O(n)$.

**Overall Runtime:**  
$$
O(n + m)
$$

which is linear in the size of the input graph.

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.8</h1>

Pouring water. We have three containers whose sizes are 10 pints, 7 pints, and 4  pints, respectively. The 7-pint and 4-pint containers start out full of water, but  the 10-pint container is initially empty. We are allowed one type of operation:  pouring the contents of one container into another, stopping only when the  source container is empty or the destination container is full. We want to know  if there is a sequence of pourings that leaves exactly 2 pints in the 7- or 4-pint  container.  

(a) Model this as a graph problem: give a precise definition of the graph  involved and state the specific question about this graph that needs to be  answered.  

(b) What algorithm should be applied to solve the problem? 

-----

(a) Let G = (V,E) be our (directed) graph. We will model the set of nodes as triples of numbers
(a0, a1, a2) where the following relationships hold: Let S0 = 10, S1 = 7,S2 = 4 be the sizes of the
corresponding containers. ai will correspond at the actual contents of the ith container. It must
hold 0 <= ai <= Si for i = 0, 1, 2 and at any given node a0 + a1 + a2 = 11 (the total amount of
water we started from). An edge between two nodes (a0, a1, a2) and (b0, b1, b2) exists if both the
following are satisfied :
- the two nodes di↵er in exactly two coordinates (and the third one is the same in both).
- if i, j are the coordinates they di↵er in, then either ai = 0 or aj = 0 or ai = Si or aj = Sj .
The question that needs to be answered is wether there exists a path between the nodes (0, 7, 4)
and (⇤, 2, ⇤) or (⇤, ⇤, 2) where ⇤ stands for any (allowed) value of the corresponding coordinate.

(b) Given the above description, it is easy to see that a DFS algorithm on that graph should be
applied, starting from node (0, 7, 4) with an extra line of code that halts and answers ’YES’ if one
of the desired nodes is reached and ’NO’ if all the connected component of the starting node is
exhausted an no desired vertex is reached.

(c) It is easy to see that after a few steps of the algorithm (depth 6 on the dfs tree) the node (2,7,2)
is reached, so we answer ‘YES’.

-----
<font color='red'><b>OA solution</b></font>

## (a) Algorithm

Represent each physically possible configuration of the three containers as a vertex of a directed graph $G = (V, E)$.
A configuration is a triple $(a_1, a_2, a_3)$ where:
	•	$a_1$ is the amount of water in the 10-pint container, $0 \le a_1 \le 10$,
	•	$a_2$ is the amount of water in the 7-pint container, $0 \le a_2 \le 7$,
	•	$a_3$ is the amount of water in the 4-pint container, $0 \le a_3 \le 4$,
	•	and the invariant $a_1 + a_2 + a_3 = 11$ always holds since the total amount of water (7 + 4) remains constant.

The starting configuration is $(0, 7, 4)$, representing the 10-pint container empty, and both the 7-pint and 4-pint containers full.

For every configuration $(a_1, a_2, a_3)$, we create edges representing legal pouring operations.
A pour operation transfers water from one container to another until either the source is empty or the destination is full.
For each ordered pair of distinct containers $i \to j$ with capacities $C = (10, 7, 4)$:

$$
\Delta = \min(a_i, C_j - a_j)
$$

We then form a new configuration where:

$$
a_i’ = a_i - \Delta, \quad a_j’ = a_j + \Delta
$$

and the remaining container’s amount stays the same.
Add a directed edge from $(a_1, a_2, a_3)$ to this new configuration.
Each vertex therefore has up to six outgoing edges (one for each possible pour direction).

We then run Breadth-First Search (BFS) starting from the vertex $(0, 7, 4)$.
BFS explores all configurations reachable from the start by sequences of pours.
We stop and return True as soon as a configuration is found with $a_2 = 2$ or $a_3 = 2$.
If BFS finishes without finding such a configuration, return False.

## (b) Justification of Correctness

Each vertex of $G$ corresponds exactly to a physically valid configuration of the containers, and each edge represents a valid single-pour operation.
Thus, any path in $G$ corresponds to a sequence of pourings possible under the problem’s rules.
BFS systematically explores all reachable configurations, layer by layer, from the start configuration $(0, 7, 4)$.

Since BFS visits every configuration reachable through any sequence of legal pours, it will find a configuration with exactly 2 pints in the 7- or 4-pint container if and only if such a configuration exists.
Therefore, returning True when such a configuration is reached, and False otherwise, correctly determines whether the goal state is achievable.

## (c) Runtime Analysis

Let $|V|$ denote the number of valid configurations and $|E|$ the number of pour transitions between them.
Each vertex can have at most 6 outgoing edges (one for each possible pour direction), so $|E| = O(|V|)$.
	•	Neighbor generation: For each configuration, generating all neighbors takes $O(1)$ time per edge, giving $O(|E|)$.
	•	BFS traversal: BFS on an adjacency-list graph runs in $O(|V| + |E|)$ time and $O(|V|)$ space.

Therefore, the total runtime is:

$$
O(|V| + |E|) = O(|V|)
$$

which is linear in the number of possible configurations.
Since the number of valid states is finite (bounded by the capacities of the containers), this algorithm runs efficiently and guarantees termination.

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.11</h1>

Design a linear-time algorithm which, given an undirected graph G and a  particular edge e in it, determines whether G has a cycle containing e. 

-----

Delete e, run DFS

-----
<font color='red'><b>OA solution</b></font>

## (a) Algorithm

Represent the undirected graph $G = (V, E)$ in adjacency-list form, and let the given edge be $e = {u, v} \in E$.
We will use Depth-First Search (DFS) as a black box to test reachability.


1.  Remove the edge $e$.
Construct a new graph $G’ = (V, E \setminus {e})$.
In implementation, we can simply ignore $e$ during traversal instead of physically removing it.

2.  Run DFS from one endpoint.
Run DFS starting from vertex $u$ in $G’$.
- If vertex $v$ is reachable from $u$, return True — this means $G$ contains a cycle that includes $e$.
- If vertex $v$ is not reachable from $u$, return False — there is no such cycle.

Thus, the algorithm determines whether there exists a cycle containing the edge $e$.

## (b) Justification of Correctness

An undirected graph contains a cycle passing through the edge $e = {u, v}$ if and only if
there exists another path between $u$ and $v$ that does not use $e$.
- ($\Rightarrow$) Suppose $G$ has a cycle that contains $e$.
Removing $e$ from the cycle leaves a $u \leadsto v$ path in $G’$, so $v$ is reachable from $u$ in $G’$.
- ($\Leftarrow$) Suppose $v$ is reachable from $u$ in $G’$.
Then there exists a $u \leadsto v$ path that does not use $e$.
Adding $e$ back creates a cycle passing through $e$ in $G$.

Hence, the algorithm correctly determines whether a cycle containing $e$ exists by checking whether $u$ and $v$ remain connected after removing $e$.

## (c) Runtime Analysis

Let $n = |V|$ and $m = |E|$.
	•	Removing (or ignoring) $e$: $O(1)$
	•	Running DFS on $G’$: Visits each vertex and edge at most once — $O(n + m)$

Therefore, the total runtime of the algorithm is:

$$
O(n + m)
$$

which is linear in the size of the input graph. The space complexity is also $O(n)$ for DFS bookkeeping.

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.16</h1>

Suppose a CS curriculum consists of n courses, all of them mandatory. The  prerequisite graph G has a node for each course, and an edge from course v to  course w if and only if v is a prerequisite for w. Find an algorithm that works  directly with this graph representation, and computes the minimum number of  semesters necessary to complete the curriculum (assume that a student can take  any number of courses in one semester). The running time of your algorithm  should be linear. 

-----

The algorithm is to always pick a source, delete it from the graph and then recurse on the resulting
graph. A source is just a vertex with indegree 0. Thus, we can find all the sources in the initial graph
by performing a DFS and computing the indegree of all the vertices. We add all the vertices with
indegree 0 to a list L.

At each step, the algorithm then removes an element from L (a source) and reduces the indegree of
each of its neighbors by 1 (this corresponds to deleting it from the graph). If this changes the indegree
of any vertex to 0, we add it to L. The removed element is assigned the next position in the ordering.

Computing all indegrees in the first step only requires a DFS which takes linear time. Subsequently,
we only look at each edge (u, v) at most once when we are removing u from L. Hence, the total time
is linear in the size of the graph.

-----
<font color='red'><b>OA solution</b></font>

## (a) Algorithm

Represent the prerequisite structure as a directed acyclic graph (DAG) $G = (V, E)$ in adjacency-list form,
where each vertex represents a course and each directed edge $(u, v) \in E$ means “course $u$ is a prerequisite for course $v$.”

The goal is to compute the minimum number of semesters required to complete all courses,
given that any number of courses can be taken in a semester as long as their prerequisites have been completed.

1.	Topological sorting.
Perform a DFS-based topological sort on $G$ to produce an ordering
$T = [T_1, T_2, \dots, T_n]$, where if $(u, v) \in E$, then $u$ appears before $v$ in $T$.

2.	Initialize semester counts.
For each vertex $v \in V$, set $\text{sem}[v] = 1$.
This means that if a course has no prerequisites, it can be taken in semester 1.

3.	Propagate semester dependencies.
Process vertices in topological order.
For each directed edge $(u, v) \in E$, update:
$$
\text{sem}[v] = \max(\text{sem}[v], \text{sem}[u] + 1)
$$
This ensures that each course $v$ is scheduled one semester after the latest of its prerequisites.

4.	Return the result.
The minimum number of semesters required to complete the curriculum is:
$$
\max_{v \in V} \text{sem}[v]
$$

## (b) Justification of Correctness

Because each course must be completed after all of its prerequisites,
the prerequisite structure naturally forms a DAG.
A topological ordering ensures that each course appears after all of its prerequisites in the sequence.

The algorithm correctly computes the earliest semester a course can be taken:
	•	Courses with no prerequisites are assigned semester 1.
	•	Each dependent course $v$ is assigned a semester number one greater than the maximum semester of any of its prerequisites.

Thus, $\text{sem}[v]$ represents the length of the longest prerequisite chain ending at $v$.
The maximum of these values corresponds to the longest path length in the DAG,
which equals the minimum number of semesters needed to complete all courses.

## (c) Runtime Analysis

Let $n = |V|$ and $m = |E|$.
	•	Topological sorting: Each vertex and edge is visited once.
Time complexity: $O(n + m)$.
	•	Semester propagation: Each edge $(u, v)$ is relaxed once.
Time complexity: $O(n + m)$.
	•	Finding the maximum semester count: $O(n)$.

Thus, the total runtime is:

$$
O(n + m) + O(n + m) + O(n) = O(n + m)
$$

The algorithm runs in linear time with respect to the size of the graph.
The space complexity is also $O(n)$ for storing semester values and DFS structures.

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.22</h1>

Give an efficient algorithm which takes as input a directed graph G = (V, E),  and determines whether or not there is a vertex s ∈ V from which all other  vertices are reachable. 

-----

Run SCC, check if there is only one source SCC

-----
<font color='red'><b>OA solution</b></font>

## (a) Algorithm

Represent the directed input graph $G = (V, E)$ in adjacency-list form.
We want to determine whether there exists a vertex $s \in V$ from which all other vertices are reachable.
We will use the Strongly Connected Components (SCC) algorithm as a black box.

1.	Compute SCCs.
Run an SCC algorithm (e.g., Kosaraju or Tarjan) on $G$ to obtain the partition ${C_1, C_2, \dots, C_k}$ of vertices into strongly connected components.
Build the corresponding condensation graph
$$
G_{\text{SCC}} = (V_{\text{SCC}}, E_{\text{SCC}})
$$
where each vertex represents an SCC, and there is an edge $(C_i, C_j)$ if there exists an edge $(u, v) \in E$ with $u \in C_i$ and $v \in C_j$.

2.	Identify source SCCs.
Initialize an array has_in of size $k$, setting all entries to False.
For every edge $(C_i, C_j) \in E_{\text{SCC}}$, mark has_in[j] = True.
After this scan, any component $C_\ell$ with has_in[ℓ] = False is a source SCC (no incoming edges).

3.	Check uniqueness.
-	If there is exactly one source SCC $C^$, return True and identify any vertex $s \in C^$ as the starting vertex from which all others are reachable.
-	If there are zero or more than one source SCCs, return False.

## (b) Justification of Correctness

The SCC condensation graph $G_{\text{SCC}}$ is a directed acyclic graph (DAG) that preserves the reachability relations between strongly connected components.
-	If there is exactly one source SCC $C^*$:
Every other component must have at least one incoming edge (since $C^$ is the only source).
Therefore, all other components are reachable from $C^$.
Any vertex $s \in C^*$ can reach all vertices in $G$ because:

1. $s$ can reach every vertex inside its own SCC (by definition of SCC), and

2. There exists a path from $C^*$ to every other SCC in the condensation graph.
-	If there are two or more source SCCs:
Vertices in different source SCCs cannot reach one another, since a source SCC has no incoming edges.
Hence, no single vertex can reach every other vertex in $G$.

Therefore, the algorithm correctly determines whether such a universal-reach vertex exists by checking for a unique source SCC.

## (c) Runtime Analysis

Let $n = |V|$ and $m = |E|$.
- SCC computation:
Using Tarjan’s or Kosaraju’s algorithm takes $O(n + m)$ time.
- Identifying source SCCs:
Scanning all edges in $E_{\text{SCC}}$ (at most one per original edge) and marking in-degree requires $O(m)$ time.
- Final check:
Scanning all SCCs to count sources requires $O(n)$ time.

Thus, the total running time is:

$$
O(n + m) + O(m) + O(n) = O(n + m)
$$

which is linear in the size of the input graph.
The space complexity is also $O(n + m)$ for storing the adjacency lists and SCC structures.

<br>
<br>
<br>
<h1 style="text-align:center; color:blue">3.24</h1>


Give a linear-time algorithm for the following task.  

Input: A directed acyclic graph G  

Question: Does G contain a directed path that touches every vertex exactly once? 

-----

Start by linearizing the DAG. Since the edges can only go in the increasing direction in the linearized
order, and the required path must touch all the vertices, we simply check if the DAG has an edge (i, i+1)
for every pair of consecutive vertices labelled i and i + 1 in the linearized order. Both, linearization
and checking outgoing edges from every vertex, take linear time and hence the total running time is
linear.

-----
<font color='red'><b>OA solution</b></font>

## (a) Algorithm

Treat the input as a directed acyclic graph $G = (V, E)$ represented in adjacency-list form.
We will use a DFS-based topological ordering as our black box algorithm.

1.	Topological sort.
Run a DFS-based topological ordering on $G$.
Let the resulting order be $T[1], T[2], \dots, T[n]$, arranged from source to sink (so if $(u, v) \in E$, then $u$ appears before $v$ in $T$).

2.	Consecutive-edge check.
For each consecutive pair $(T[i], T[i+1])$ where $i = 1, \dots, n-1$, check whether the edge $(T[i], T[i+1])$ exists in $E$.
	•	If any such edge is missing, return False.
	•	If all consecutive pairs are edges, return True, since the path

$$
T[1] \rightarrow T[2] \rightarrow \cdots \rightarrow T[n]
$$

visits every vertex exactly once.

## (b) Justification of Correctness

If the algorithm returns True, then every consecutive pair in the topological order $T$ is connected by a directed edge.
Thus, the sequence $T[1] \rightarrow T[2] \rightarrow \cdots \rightarrow T[n]$ forms a directed path that touches every vertex exactly once.

If the algorithm returns False, then some consecutive pair $(T[i], T[i+1])$ in the topological order lacks an edge between them.
In that case, it is impossible to have a single directed path that touches every vertex once in the order $T$.
In a DAG, any Hamiltonian (single spanning) path must appear as a topological order where every consecutive pair is connected by an edge, so failing this check proves no such path exists.

Because the DFS-based topological sort guarantees that all edges go from earlier to later vertices in $T$, this approach correctly detects whether $G$ has a directed path that touches every vertex exactly once.

## (c) Runtime Analysis
	•	Topological sort (DFS-based): Each vertex is visited once and each edge is examined a constant number of times, taking $O(n + m)$ time.
	•	Consecutive-edge check: There are $n - 1$ pairs $(T[i], T[i+1])$.
Checking whether $(T[i], T[i+1]) \in E$ by scanning adjacency lists costs time proportional to the out-degree of each vertex. Summed over all vertices, this is $O(m)$.

Thus, the total runtime is:

$$
O(n + m) + O(n + m) = O(n + m)
$$

which is linear in the size of the input graph.